In [ ]:
options(jupyter.rich_display = F)

# TRAVELLING SALESMAN

**by Serhat Çevikel**

We will implement a simple "greedy" solution to travelling salesman problem: The shortest route to take to visit all cities in a path

We will compare the solution to a brute force approach (trying all permutations) and see whether the fast, simple approach is as good as the slow brute force approach

Of course, we will take advantage of the expressiveness of R language: Do much with little

First let's download the distance matrix for 81 city centers again: We have a 81x81 matrix of bird fly distances in km's between 81 province centers in Turkey. To retrieve this matrix please follow the link below to download the file distance.RData:

[distance.RData](../file/distance.RData)

In [ ]:
options(jupyter.rich_display = F)

Load the data:

In [ ]:
load("~/file/distance2.RData")

See what is inside:

In [ ]:
distance2[1:5, 1:5]

Now let's have a sample of the cities:

In [ ]:
cities <- c("istanbul", "adana", "ankara", "van", "mugla", "artvin")
#cities <- c("istanbul", "adana", "ankara", "van", "mugla", "artvin", "kayseri", "usak", "erzincan")
#cities <- c("istanbul", "adana", "ankara", "van", "mugla", "artvin", "kayseri", "usak", "erzincan", "sinop")
cities

## Greedy approach

**Exercise 1:**

Write a function shortest() using the below template (just fill in ...'s)

The logic is:

- We select a starting position out of the cities
- We keep track of visited and unvisited cities and the last visited location
- In each iteration, we select the next city with the shortest distance to the current one
- We continue until we run out of unvisited cities
- We report both the distance and the vector of cities in visited order as a list

```R

shortest <- function(path = cities, start = "istanbul", dist = distance2)
{
    dist2 <- distance2[cities, cities] # subset the larger matrix so that we deal with a smaller one
    diag(dist2) <- Inf # toggle the diagonal to Inf so that min() does not return 0's

    # we keep track of two city vectors: path is for unvisited cities, path2 is for visited ones
    path <- setdiff(path, start) # delete the starting city
    path2 <- start # append the start city to visited

    location <- start # current location is the start city
    distance <- 0 # initiate the cumulative distance
    
    while(length(path) > 0) # as long as unvisited cities exist
    {
        rowx <- ... # get the row for current location, columns for unvisited cities
        nextind <- ... # get the index of the minimum distance to next city, use which.min
        distance <- ... # add the minimum distance to the cumulative
        location <- ... # get the next location
        path <- ... # delete the next city from unvisited ones, use setdiff
        path2 <- ... # append the next city to visited ones
    }
    
    return(list(..., ...)) # report the cumulative distance and cities in visited order as a list
}

```

**Solution:**

In [ ]:
shortest <- function(path = cities, start = "istanbul", dist = distance2)
{
    dist2 <- distance2[cities, cities] # subset the larger matrix so that we deal with a smaller one
    diag(dist2) <- Inf # toggle the diagonal to Inf so that min() does not return 0's

    # we keep track of two city vectors: path is for unvisited cities, path2 is for visited ones
    path <- setdiff(path, start) # delete the starting city
    path2 <- start # append the start city to visited

    location <- start # current location is the start city
    distance <- 0 # initiate the cumulative distance
    
    while(length(path) > 0) # as long as unvisited cities exist
    {
        rowx <- dist2[location,path] # get the row for current location, columns for unvisited cities
        nextind <- which.min(rowx) # get the index of the minimum distance to next city
        distance <- distance + min(rowx) # add the minimum distance to the cumulative
        location <- path[nextind] # get the next location
        path <- setdiff(path, location) # delete the next city from unvisited ones
        path2 <- c(path2, location) # append the next city to visited ones
    }
    
    return(list(distance, path2)) # report the cumulative distance and cities in visited order as a list
}

Now let's check for different starting cities:

In [ ]:
shortest(start = "istanbul")
shortest(start = "ankara")

Note that, total distance differs for the starting city. So for the optimal solution, we should also find the optimal starting city, along with the optimal path for a given starting city

**Exercise 2:**

What if we try the function for all possible starting cities as such (fill in ... part):

```R
alternatives <- lapply(cities, function(x) ...)
```

**Solution 2:**

In [ ]:
alternatives <- lapply(cities, function(x) shortest(cities, x))

Now we list the optimal paths for all alternative starting cities:

In [ ]:
alternatives

**Exercise 3:**

Now let's choose the shortest one programmatically as such:

```R
dist1 <- sapply(alternatives, ...)
alternatives[[...]]

[[1]]
[1] 2219

[[2]]
[1] "mugla"    "istanbul" "ankara"   "adana"    "van"      "artvin"
```

**Solution 3:**

In [ ]:
dist1 <- sapply(alternatives, "[[", 1)
dist1
alternatives[[which.min(dist1)]]

For a given vector of n cities, we just construct and test n permutations. Now let's compare with the brute force or exhaustive approach

## BRUTE FORCE APPROACH

For a given vector of size n, following implementation of mine creates all n! distinct permutations as a matrix: 

In [ ]:
perms <- function(vec = 1:5, carry = NULL)
{
    if (length(vec) == 1)
    {
        return(c(carry, vec))
    }
    else
    {
        listx <- lapply(vec, function(x) perms(setdiff(vec, x), c(carry, x)))
        return(do.call(rbind, listx))
    }
}

Note that my implementation is identical in terms of the algorithm to this one in stackoverflow that I saw ater:

https://stackoverflow.com/a/29023189

As the proverb goes: "Aklın yolu birdir"

Check the number of permutations and some of them:

In [ ]:
dim(perms(1:5))
head(perms(1:5))

Now another efficient and expressive implementation of the previous take me home question: The total distance of a path of cities

In [ ]:
distancex <- function(path = cities, dist = distance2)
{
    mat <- cbind(path[-length(path)], path[-1])
    sum(distance2[mat])
}

The idea here is that, to subset non-contiguous cells of a matrix of any size, subset it with a two column matrix in which:
- first column shows the row indices (or colnames) of the cells
- second column shows the column indices (or rownames) of the cells

For the original order of the cities the total distance is:

In [ ]:
distancex(cities)

Now let's generate all 6! = 720 distinct permutations of the six cities in our vector:

In [ ]:
permsx <- perms(cities)
dim(permsx)
head(permsx)

**Exercise 4:**

Using apply(), distancex(), min() and which.min() functions and permsx matrix, find the distances of all permutations, and select the shortest distance and the path

Is the solution same as the one we found in the greedy approach?

**Solution 4:**

In [ ]:
distx <- apply(permsx, 1, distancex)
min(distx)
permsx[which.min(distx),]

Trying 6 versus 720 paths! Greedy approach shines in longer paths